In [1]:
import numpy as np
import MDAnalysis as md
import torch
from shapeGMMTorch import ShapeGMM, utils
import matplotlib.pyplot as plt
import pickle

# Read MD Trajectory

In [2]:
prmtopFile = "diala_vacuum.parm7"
trajFile1 = "adp_vac.dcd"
coord = md.Universe(prmtopFile,trajFile1)
print("Number of atoms in file:", coord.atoms.n_atoms)
print("Number of frames in trajectory:", coord.trajectory.n_frames)
# select all atoms except methyl hydrogen atoms
atomSel = coord.select_atoms("index 1 4 5 6 7 8 9 10 14 15 16 17 18")
print(atomSel.names)
traj_pos = np.empty((coord.trajectory.n_frames,atomSel.n_atoms, 3))
for ts in coord.trajectory:
    traj_pos[ts.frame] = atomSel.positions

Number of atoms in file: 22
Number of frames in trajectory: 50000
['CH3' 'C' 'O' 'N' 'H' 'CA' 'HA' 'CB' 'C' 'O' 'N' 'H' 'C']


# Fit ShapeGMM Kronecker 3-state model

In [3]:
device = torch.device("cpu")
dtype = torch.float64
sgmm3_kronecker = utils.sgmm_fit_with_attempts(traj_pos[::100], 3, 15, dtype=dtype, device=device)

Number of components    : 3
Number of attempts      : 15
Covariance type         : kronecker
Data type (dtype)       : torch.float64
Device                  : cpu
Number of train frames  : 500
Number of atoms         : 13
Init Component Method   : kmeans++
Random seed             : 1234
 Attempt  Log Like per Frame Wallclock Time (s)
------------------------------------------------------------
       1             78.6182           2.885
       2             78.0696           0.308
       3             90.9848           0.307
       4             92.1010           0.245
       5             78.8200           0.333
       6             78.7293           0.279
       7             92.0211           0.348
       8             78.7098           0.282
       9             92.0336           0.261
      10             78.7801           0.336
      11             78.5056           0.501
      12             92.0434           0.331
      13             78.4184           0.214
      14          

# Create New GMM Object with Minimized Means and Calculated Hessian

In [4]:
import rgShapeGMM

In [9]:
remove_indeces = [0,2,3,11,12,13,19,20,21]
parm_file_name = "diala_vacuum.parm7"
adp_sgmm3_min_center_hess_gmm3n = rgShapeGMM.refine_gmm(sgmm3_kronecker,parm_file_name, remove_indeces)

Reading system metadata from prmtop file
Number of atoms from prmtop file: 22
Number of atom types from prmtop file: 7
Number of bonds containing hydrogens: 12
Number of bonds NOT containing hydrogens: 9
Number of angles containing hydrogens: 25
Number of angles NOT containing hydrogens: 11
Number of dihs containing hydrogens: 43
Number of dihs NOT containing hydrogens: 24
Length of excluded atoms list: 99
Number of unique bond types: 11
Number of unique angle types: 21
Number of unique dih types: 14
         Current function value: -7.718508
         Iterations: 108
         Function evaluations: 191
         Gradient evaluations: 181
Bond energy: 0.30187844609002634
Angle energy: 1.0966371106362203
Dihedral energy: 0.10909875934000049
Electrostatic energy: -60.097281626262074
VDW energy: -0.7778628025699545
Scaled 1-4 vdw: 1.8543623659537327
Scaled 1-4 electrostatics: 48.956780571591445
CMAP energy: 0.8378796256043167
Total energy: -7.71850754961629
         Current function value: -

In [10]:
# save object
fileObj = open("adp_sgmm3_min_center_hess_gmm3n.obj",'wb')
pickle.dump(adp_sgmm3_min_center_hess_gmm3n,fileObj)
fileObj.close()